In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("movie_analysis_with_names").master("local[1]").getOrCreate()

ModuleNotFoundError: No module named 'pyspark'

In [25]:
import codecs

def loadMovieNames():
    movieNames = {}
    with codecs.open("./ml-100k/u.item",encoding = "ISO-8859-1", errors = "ignore") as f:
        for line in f:
            fields = line.split('|')
            movieNames[int(fields[0])] = fields[1]
    return movieNames

nameDict = spark.sparkContext.broadcast(loadMovieNames())


In [26]:
from pyspark.sql.types import IntegerType, LongType, StructField, StructType

schema = StructType([
    StructField("user_id", IntegerType(), True),
    StructField("movie_id", IntegerType(), True),
    StructField("rating", IntegerType(), True),
    StructField("timestamp", LongType(), True)
])

movies_data = spark.read.csv("./ml-100k/u.data", sep="\t", schema= schema)

In [27]:
from pyspark.sql.functions import count, udf, desc

movie_counts= movies_data.groupBy("movie_id").agg(count("movie_id").alias("count"))
movie_counts.show()

+--------+-----+
|movie_id|count|
+--------+-----+
|     496|  231|
|     471|  221|
|     463|   71|
|     148|  128|
|    1342|    2|
|     833|   49|
|    1088|   13|
|    1591|    6|
|    1238|    8|
|    1580|    1|
|    1645|    1|
|     392|   68|
|     623|   39|
|     540|   43|
|     858|    3|
|     737|   59|
|     243|  132|
|    1025|   44|
|    1084|   21|
|    1127|   11|
+--------+-----+
only showing top 20 rows



In [28]:
def lookupName(movieId):
    return nameDict.value[movieId]

lookupNameUDF = udf(lookupName)

In [29]:
movie_with_names = movie_counts.withColumn("Title", lookupNameUDF(movie_counts["movie_id"])).orderBy(desc("count"))

In [23]:
import sys
print(sys.executable)

/Library/Developer/CommandLineTools/usr/bin/python3
